In [4]:
%matplotlib inline
import pandas as pd
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [6]:
DATA_HUB['kaggle_house_train'] = (  #@save
    DATA_URL + 'kaggle_house_pred_train.csv',
    '585e9cc93e70b39160e7921475f9bcd7d31219ce')

DATA_HUB['kaggle_house_test'] = (  #@save
    DATA_URL + 'kaggle_house_pred_test.csv',
    'fa19780a7b011d9b009e8bff8e99922a8ee2eb90')

In [7]:
train_data = pd.read_csv(download('kaggle_house_train'))
test_data = pd.read_csv(download('kaggle_house_test'))

正在从http://d2l-data.s3-accelerate.amazonaws.com/kaggle_house_pred_train.csv下载..\data\kaggle_house_pred_train.csv...
正在从http://d2l-data.s3-accelerate.amazonaws.com/kaggle_house_pred_test.csv下载..\data\kaggle_house_pred_test.csv...


In [8]:
train_data.shape, test_data.shape
# 训练数据集包括1460个样本，每个样本80个特征和1个标签
# 而测试数据包含1459个样本，每个样本80个特征。

((1460, 81), (1459, 80))

In [9]:
# 让我们看看前四个和最后两个特征，以及相应标签（房价）。
train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]]

,Id,MSSubClass,MSZoning,LotFrontage,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,WD,Normal,208500
1,2,20,RL,80.0,WD,Normal,181500
2,3,60,RL,68.0,WD,Normal,223500
3,4,70,RL,60.0,WD,Abnorml,140000


In [11]:
# id对于预测无用，删掉。将训练与测试所有特征都放进来
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

In [12]:
# 数字特征进行标准化、处理缺失值
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [13]:
# 将非数字特征转化为one-hot编码
# Dummy_na=True 将“na”（缺失值）视为有效的特征值，并为其创建指示符特征。
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape

(2919, 331)

In [14]:
n_train = train_data.shape[0] # 得到训练数据个数
train_features = torch.tensor(all_features[:n_train].values, dtype=d2l.float32)
test_features = torch.tensor(all_features[n_train:].values, dtype=d2l.float32)
train_labels = torch.tensor(    # 不懂这一句
    train_data.SalePrice.values.reshape(-1, 1), dtype=d2l.float32)

In [15]:
loss = nn.MSELoss()
in_features = train_features.shape[1]   # 有多少种特征

def get_net():
    net = nn.Sequential(nn.Linear(in_features, 1))
    return net

In [16]:
# 我们不关心预测误差的绝对值，而是关心预测误差的相对值，一个做法是取log
def log_rmse(net, features, labels):
    # 为了在取对数时进一步稳定该值，将小于1的值设置为1
    clipped_preds = torch.clamp(net(features), 1, float('inf'))
    rmse = torch.sqrt(loss(torch.log(clipped_preds), torch.log(labels)))
    return rmse.item()

In [1]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_deacy, batch_size):
    train_ls, test_ls = [], []
    train_iter = d2l.load_array((train_features, train_labels), batch_size)
    # 不使用SGD，而是使用Adam优化算法，对学习率不敏感
    updater = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay=weight_deacy)

    for epoch in range(num_epochs):
        for X, y in train_iter:
            l = loss(net(X), y)
            updater.zero_grad()
            l.backward()
            updater.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls